In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [44]:
%%html
<style>
.nbviewer div.output_area {
  overflow-y: auto;
  max-height: 300px; /* or value of your choosing */
}
</style>

In [6]:
import pandas as pd
import sys
from Analysis.KS_hotlips_analysis_tools_v4 import *

import gzip
import gc
import warnings
import tables

In [34]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
etf_sym_lst=['069500.KS', '278530.KS', '102110.KS', '267770.KS', '252710.KS', '123310.KS']

In [7]:
sDate = pd.datetime(2018, 6, 1)
eDate = pd.datetime(2019, 1, 22)

## __Approach__
***
Get all trades and order operations from Hotlips logs between Jun 2018 and Jan 20

Pickoff trades are identified by filtering for trades 
where the executed price is different from the order's last target update price.

I have included only the most liquid names that we trade, ie KODEX200 and all of TIGER names.

## __Purpose__
***
We aim to use this info to link back and see what the future book / base quote was doing at the time.
Construt timeseries model on the fut order book to refine our base quote.

Spot inefficiencies / ways to improve the qouting logic.

As an example, we already know that currently Hotlips would processed queued u pamend operations in sequence, rather than jumping to the latest target update level. 

In the analysis below, we have seen many cases of this when there's two amend attempts in quick succession, the order gets taken out at the target level of the first amend.

## __Daily Symbol Summary__
***
*all_trades_$notional_sum*: total dollar notional traded on the day

*po_$notional_sum*: total dollar notional of trades that are classified as pickoff

*pickoff_$notional_pct*: percentage of trades that are pickoffs weighted by dollar notional.

*wgt_avg_px_diff*: the weighted average difference between target and executed price
    The symbols we trade have a tick size of 5.
    So, a wgt avg diff of -10 would indicate, on average, we got picked off by 2 ticks in the sym on the day.

*est_pnl*: wgt_avg_px_diff * total number of shrs traded in the symbol

Note: If you want to see the output in the expanded view, scrolling toggled on/off by clicking on the white space to the left of the output table.

__069500.KS KODEX200__

In [18]:
y.gen_summary_by_sym(sym='069500.KS', ret_df_style=True)

,est_pnl,wgt_avg_px_diff,pickoff_$notional_pct,po_$notional_sum,no_of_pickoff,no_of_trades,all_trades_$notional_sum
2018-06-01,nan,nan,nan%,nan,0,0,nan
2018-06-04,nan,nan,nan%,nan,0,0,nan
2018-06-05,nan,nan,nan%,nan,0,0,nan
2018-06-06,nan,nan,nan%,nan,0,0,nan
2018-06-07,nan,nan,nan%,nan,0,0,nan
2018-06-08,nan,nan,nan%,nan,0,0,nan
2018-06-11,-333,-6.1,39%,"2,921,886",53,305,"7,583,545"
2018-06-12,-403,-7.3,46%,"2,695,619",34,248,"5,816,388"
2018-06-13,nan,nan,nan%,nan,0,0,nan
2018-06-14,-123,-5.4,24%,"1,353,629",28,294,"5,606,845"


__102110.KS TIGER200__

In [20]:
y.gen_summary_by_sym('102110.KS', ret_df_style=True)

,est_pnl,wgt_avg_px_diff,pickoff_$notional_pct,po_$notional_sum,no_of_pickoff,no_of_trades,all_trades_$notional_sum
2018-06-01,-255,-5.2,45%,"2,953,136",112,877,"6,579,071"
2018-06-04,-34,-5.0,52%,"423,040",8,41,"808,257"
2018-06-05,-184,-5.3,57%,"2,102,816",30,114,"3,671,807"
2018-06-06,nan,nan,nan%,nan,0,0,nan
2018-06-07,-257,-6.0,45%,"2,305,614",36,186,"5,128,562"
2018-06-08,-622,-6.0,57%,"5,618,959",53,155,"9,824,942"
2018-06-11,-172,-5.4,48%,"1,873,383",75,688,"3,898,063"
2018-06-12,-141,-5.0,50%,"1,791,495",23,256,"3,611,861"
2018-06-13,nan,nan,nan%,nan,0,0,nan
2018-06-14,-56,-5.6,15%,"568,234",14,169,"3,705,532"



__267770.KS TIGER Lev 2x__

In [21]:
y.gen_summary_by_sym('267770.KS', ret_df_style=True)

,est_pnl,wgt_avg_px_diff,pickoff_$notional_pct,po_$notional_sum,no_of_pickoff,no_of_trades,all_trades_$notional_sum
2018-06-01,-181,-5.0,61%,"870,418",31,66,"1,424,973"
2018-06-04,-36,-5.0,96%,"174,742",2,5,"182,510"
2018-06-05,-103,-5.0,82%,"501,905",9,14,"610,240"
2018-06-06,nan,nan,nan%,nan,0,0,nan
2018-06-07,-81,-10.0,20%,"135,166",1,14,"668,214"
2018-06-08,nan,nan,nan%,nan,0,0,nan
2018-06-11,-204,-5.0,42%,"1,008,829",23,61,"2,395,177"
2018-06-12,-252,-5.7,62%,"968,875",20,54,"1,550,838"
2018-06-13,nan,nan,nan%,nan,0,0,nan
2018-06-14,-477,-5.3,55%,"2,028,908",30,230,"3,665,515"


__252710.KS TIGER Inv 2x__

In [23]:
y.gen_summary_by_sym('252710.KS', ret_df_style=True)

,est_pnl,wgt_avg_px_diff,pickoff_$notional_pct,po_$notional_sum,no_of_pickoff,no_of_trades,all_trades_$notional_sum
2018-06-01,-94,-5.0,54%,"239,280",1,41,"446,486"
2018-06-04,-79,-5.0,31%,"195,461",13,22,"623,090"
2018-06-05,-5,-5.0,9%,"12,900",1,9,"145,862"
2018-06-06,nan,nan,nan%,nan,0,0,nan
2018-06-07,-68,-5.0,41%,"163,831",6,28,"396,271"
2018-06-08,-210,-5.0,49%,"518,006",19,113,"1,064,824"
2018-06-11,-133,-5.0,14%,"324,881",11,59,"2,307,133"
2018-06-12,-86,-5.0,41%,"208,418",10,76,"513,999"
2018-06-13,nan,nan,nan%,nan,0,0,nan
2018-06-14,-455,-5.0,32%,"1,138,962",13,361,"3,552,718"


__123310.KS Tiger Inv 1x__

In [25]:
y.gen_summary_by_sym('123310.KS', ret_df_style=True)

,est_pnl,wgt_avg_px_diff,pickoff_$notional_pct,po_$notional_sum,no_of_pickoff,no_of_trades,all_trades_$notional_sum
2018-06-01,-0,-5.0,0%,71,3,182,"75,709"
2018-06-04,-1,-5.0,4%,"2,901",3,30,"72,705"
2018-06-05,-22,-5.0,57%,"59,849",12,46,"105,692"
2018-06-06,nan,nan,nan%,nan,0,0,nan
2018-06-07,-15,-5.1,22%,"37,883",15,90,"169,406"
2018-06-08,0,0.0,0%,0,0,28,"130,582"
2018-06-11,-15,-5.0,15%,"39,743",12,63,"258,099"
2018-06-12,-33,-5.0,59%,"87,763",4,32,"149,784"
2018-06-13,nan,nan,nan%,nan,0,0,nan
2018-06-14,-77,-8.3,23%,"89,397",4,100,"383,158"


## __Period pickoff pnl summary__
***
This is used for both overview and a pointer to look into days with the biggest pickoffs.

In [42]:
y.gen_summary_by_field(sDate=sDate, eDate=eDate, fld='est_pnl', ret_df_style=True)

,069500.KS,278530.KS,102110.KS,267770.KS,252710.KS,123310.KS
2018-06-01,nan,-0,-255,-181,-94,-0
2018-06-04,nan,-1,-34,-36,-79,-1
2018-06-05,nan,0,-184,-103,-5,-22
2018-06-06,nan,nan,nan,nan,nan,nan
2018-06-07,nan,-1,-257,-81,-68,-15
2018-06-08,nan,-30,-622,nan,-210,0
2018-06-11,-333,-149,-172,-204,-133,-15
2018-06-12,-403,-176,-141,-252,-86,-33
2018-06-13,nan,nan,nan,nan,nan,nan
2018-06-14,-123,-205,-56,-477,-455,-77


Using 069500.KS from 2018-10-11 as an example: on the day we saw that pickoff pnl was -1,612.

We look up all of 069500.KS' pickoff trades on the day as below:

In [27]:
td = pd.datetime(2018, 10, 11)

In [28]:
x = ks_hotlips_log_ute(td=td, scope_name='KOSPI', etf_sym_lst=etf_sym_lst, fut_sym='KMH9', fx_rate=1130.0, gen_order_info_d=True)

In [29]:
x.loop_process_orders_per_sym()

In [35]:
x.sym_order_trade_info_d['069500.KS']['po_trades_df']

,ts,side,trade_$notional,tgt_trade_px_diff,trade_qty,trade_px,tgt_bp0,tgt_ap0,tgt_qty,tgt_px,order_state,order_tag,o_hist_df_idx,wgt
0,09:00:16.0,SELL,3567.407080,-10.0,143.0,28190.0,28175.0,28200.0,6000.0,28190.0,,4411,7,0.000294
1,09:00:16.0,SELL,2270.570796,-5.0,91.0,28195.0,28175.0,28200.0,6000.0,28190.0,,4411,5,0.000187
2,09:00:16.9,SELL,24.955752,-5.0,1.0,28200.0,28175.0,28205.0,5527.0,28200.0,,4486,4,0.000002
3,09:00:17.0,SELL,99.823009,-5.0,4.0,28200.0,28175.0,28205.0,5527.0,28200.0,,4486,7,0.000008
4,09:00:17.1,SELL,199.646018,-5.0,8.0,28200.0,28175.0,28205.0,5527.0,28200.0,,4486,9,0.000016
5,09:00:17.6,SELL,199.646018,-5.0,8.0,28200.0,28175.0,28205.0,5527.0,28200.0,,4486,12,0.000016
6,09:00:17.7,SELL,748.672566,-5.0,30.0,28200.0,28175.0,28205.0,5527.0,28200.0,,4486,14,0.000062
7,09:00:17.8,SELL,748.672566,-5.0,30.0,28200.0,28175.0,28205.0,5527.0,28200.0,,4486,16,0.000062
8,09:00:17.9,SELL,748.672566,-5.0,30.0,28200.0,28175.0,28205.0,5527.0,28200.0,,4486,18,0.000062
9,09:00:27.5,SELL,225.000000,-5.0,9.0,28250.0,28225.0,28255.0,4825.0,28250.0,,5029,5,0.000019



On row 271, wee see a pickoff trade for our full quote size of 6,000 shrs at 5 won bad.

We look into the order history of that particular order by its order tag '415625'.

This will give us colour on what was happening with the system around the time of the trade

In [41]:
pd.DataFrame(x.sym_order_trade_info_d['069500.KS']['s1_info_by_order_tags']['418656']['order_hist_a'])

,idx,ts,action,response,side,order_state,order_px,order_qty,tgt_px,tgt_qty,theo_bid,theo_ask,tgt_bp0,tgt_ap0,trade_px,trade_qty,trade_$notional,cum_traded_qty,tgt_trade_px_diff,pickoff_iv,order_tag
0,0,14:06:02.9,Update_QT,,,,NaN,NaN,NaN,NaN,27850.0,27860.0,27850.0,27865.0,NaN,NaN,NaN,NaN,NaN,False,
1,1,14:06:02.9,INSERT,,BUY,,NaN,NaN,27850.0,6000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,
2,2,14:06:02.9,,INSERTED,,ACTIVE,27850.0,6000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,
3,3,14:06:02.9,Update_QT,,,,NaN,NaN,NaN,NaN,NaN,27857.3,NaN,27860.0,NaN,NaN,NaN,NaN,NaN,False,
4,4,14:06:02.9,DELETE,,BUY,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,
5,5,14:06:02.9,Update_QT,,,,NaN,NaN,NaN,NaN,27848.6,27859.5,27845.0,27860.0,NaN,NaN,NaN,NaN,NaN,False,
6,6,14:06:02.9,,TRADE,,,NaN,NaN,27850.0,6000.0,NaN,NaN,27845.0,27860.0,27850.0,6000.0,147876.106195,NaN,-5.0,True,
7,7,14:06:02.9,,RES_Order,,FILLED,27850.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,
8,8,14:06:02.9,,,,FILLED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,
9,9,14:06:02.9,,REJECTED,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,
